In [1]:
%pip install opencv-python ultralytics


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------- ----------------------------- 10.5/39.5 MB 54.2 MB/s eta 0:00:01
   ------------------------ --------------- 24.1/39.5 MB 72.7 MB/s eta 0:00:01
   -------------------------- ------------- 26.0/39.5 MB 42.3 MB/s eta 0:00:01
   ------------------------------------- -- 37.2/39.5 MB 47.0 MB/s eta 0:00:01
   ---------------------------------------  39.3/39.5 MB 41.6 MB/s eta 0:00:01
   ---------------------------------------- 39.5/39.5 MB 37.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/922.2 kB ? eta -:--:--
   --------------------------------------- 922.2/922.2 kB 38.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 8.1/8.1 MB 51.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -------------------------------------- - 12.1/12.6 MB 58.2 MB/s eta 0:00:01
   -------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] T

In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture("classroom.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("output_video.avi", cv2.VideoWriter_fourcc(*"XVID"), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    detections = results[0]

    frame = detections.plot()

    cv2.imshow("Detection", frame)

    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 4 persons, 8 chairs, 2 dining tables, 2 tvs, 30.2ms
Speed: 1.8ms preprocess, 30.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 8 chairs, 2 dining tables, 2 tvs, 31.8ms
Speed: 1.1ms preprocess, 31.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 8 chairs, 2 dining tables, 2 tvs, 27.9ms
Speed: 1.2ms preprocess, 27.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 8 chairs, 2 dining tables, 2 tvs, 22.4ms
Speed: 1.3ms preprocess, 22.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 8 chairs, 2 dining tables, 2 tvs, 27.1ms
Speed: 1.1ms preprocess, 27.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 9 chairs, 2 dining tables, 2 tvs, 24.1ms
Speed: 1.1ms preprocess, 24.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 8 chairs, 2

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture("classroom.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("output_video.avi", cv2.VideoWriter_fourcc(*"XVID"), fps, (frame_width, frame_height))

frame_count = 0  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1 

    results = model(frame)

    detections = results[0]

    num_objects = len(detections.boxes) 

    cv2.putText(frame, f"Objects: {num_objects}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.putText(frame, f"FPS: {fps:.2f}", (frame_width - 150, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    frame = detections.plot()

    cv2.imshow("Detection", frame)

    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Total frames processed: {frame_count}")



0: 384x640 4 persons, 8 chairs, 2 dining tables, 2 tvs, 73.9ms
Speed: 8.0ms preprocess, 73.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 8 chairs, 2 dining tables, 2 tvs, 24.3ms
Speed: 1.2ms preprocess, 24.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 8 chairs, 2 dining tables, 2 tvs, 22.1ms
Speed: 1.1ms preprocess, 22.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 8 chairs, 2 dining tables, 2 tvs, 22.5ms
Speed: 1.2ms preprocess, 22.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 8 chairs, 2 dining tables, 2 tvs, 24.4ms
Speed: 1.5ms preprocess, 24.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 9 chairs, 2 dining tables, 2 tvs, 24.8ms
Speed: 1.3ms preprocess, 24.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 8 chairs, 2